In [ ]:
import os
import cv2
import random
from tqdm import tqdm
import numpy as np

source_dataset = r"isl_dataset-10"  
target_dataset = r"isl_inverted"         # Augmented dataset output folder

# Create target directory if not exists
if not os.path.exists(target_dataset):
    os.makedirs(target_dataset)

def vary_brightness(image, factor_range=(0.5, 1.5)):
    """Adjust the brightness of the image by a random factor."""
    factor = random.uniform(*factor_range)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv = hsv.astype('float32')
    hsv[:, :, 2] *= factor
    hsv[:, :, 2] = np.clip(hsv[:, :, 2], 0, 255)
    hsv = hsv.astype('uint8')
    bright_img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return bright_img


def random_rotate(image, angle_range=(-10, 10)):
    """Rotate image by a random angle within given range."""
    h, w = image.shape[:2]
    angle = random.uniform(*angle_range)
    matrix = cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1)
    rotated = cv2.warpAffine(image, matrix, (w, h), borderMode=cv2.BORDER_REFLECT)
    return rotated

def augment_and_save(image_path, save_dir, image_name):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to load image: {image_path}")
        return False

    base, ext = os.path.splitext(image_name)

    # Save Original Image
    orig_path = os.path.join(save_dir, f"{base}{ext}")
    cv2.imwrite(orig_path, img)

        # Save Brightness-Adjusted Image
    bright_img = vary_brightness(img)
    bright_name = f"{base}_bright{ext}"
    bright_path = os.path.join(save_dir, bright_name)
    cv2.imwrite(bright_path, bright_img)


    # Save Flipped Image
    flipped_img = cv2.flip(img, 1)

    # Save Flipped + Bright Image
    flipped_bright_img = vary_brightness(flipped_img)
    flipped_bright_name = f"{base}_flipped_bright{ext}"
    flipped_bright_path = os.path.join(save_dir, flipped_bright_name)
    cv2.imwrite(flipped_bright_path, flipped_bright_img)


    # Save Randomly Rotated Image
    rotated_img = random_rotate(img)

    # Save Randomly Rotated + Bright Image
    rotated_bright_img = vary_brightness(rotated_img)
    rotated_bright_name = f"{base}_rotated_bright{ext}"
    rotated_path = os.path.join(save_dir, rotated_bright_name)
    cv2.imwrite(rotated_path, rotated_bright_img)

    # Save Flipped + Rotated Image
    flipped_rotated_img = random_rotate(flipped_img)
    flipped_rotated_name = f"{base}_flipped_rotated{ext}"
    flipped_rotated_path = os.path.join(save_dir, flipped_rotated_name)
    cv2.imwrite(flipped_rotated_path, flipped_rotated_img)

    return True

# Walk through dataset
for split in ["train", "valid", "test"]:
    split_dir = os.path.join(source_dataset, split)
    target_split_dir = os.path.join(target_dataset, split)

    for class_name in os.listdir(split_dir):
        class_dir = os.path.join(split_dir, class_name)
        target_class_dir = os.path.join(target_split_dir, class_name)
        os.makedirs(target_class_dir, exist_ok=True)

        image_files = [f for f in os.listdir(class_dir) if f.lower().endswith((".jpg", ".png", ".jpeg"))]
        print(f"\nAugmenting {split}/{class_name} ({len(image_files)} images)")

        for img_name in tqdm(image_files, desc=f"{split}/{class_name}"):
            src_img_path = os.path.join(class_dir, img_name)
            augment_and_save(src_img_path, target_class_dir, img_name)

print("\nAugmentation complete. Full dataset saved to:")
print(target_dataset)
